# Trends_AC_linmix



In [40]:
import linmix
import numpy as np 
from scipy.optimize import minimize
from scipy import optimize
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.table import Table
from astropy.table import vstack
from astropy.table import Column
from scipy.optimize import minimize
from astroML.linear_model import TLS_logL
import os
import scipy
rcParams['ytick.labelsize'] = 15
rcParams['xtick.labelsize'] = 15
rcParams['axes.labelsize'] = 20
rcParams['axes.linewidth'] = 2
rcParams['font.size'] = 15
rcParams['axes.titlesize'] = 18



$\log_{10}{(f)} = A + B \log_{10}\left(\frac{\lambda_{RF}}{4000  }\right) + C (M_{i} + 23) $

with $f = \tau_{RF}$ or $f = SF_{\infty}$

B= 0.17 and - 0.479 , respectively. 

Fit with linmix  Chelsea's r-band against Shen2011. Use the same file as what we use for IDL... 

In [33]:
#sigma_hat = np.power(10,s82['log10sigmahat']) # sqrt-years 

#s82[f]['SFinf']  = np.power(10,s82[f]['log10sigmahat']) * \
 #                 np.sqrt(np.power(10,s82[f]['log10tau']) / 365.)  
    
#sfinf = sigma_hat * np.sqrt(s82['tauMAP_chelsea'] / 365.)

In [31]:
# Three-way match : Celerite x DBQSO x Shen2011 x Chelsea r-band 
name = '190409_Celerite_real_Jeff1_Shen2008-2011_s82drw_r.txt'
s82 = Table.read(name, format='ascii')

# Fit Celerite SDSS or SDSS-PS1 against Shen2011 
s82['tauMAP_chelsea'] = np.power(10,s82['log10tau']) # obs frame  in days 
s82['sigmaMAP_chelsea']  = s82['SFinf'] / np.sqrt(2) # in mags 

m2 = s82['Plike'] - s82['Pnoise'] > 2 
m3 = s82['Plike'] - s82['Pinf'] > 0.05 
m4 = s82['edge'] == 0 

m = m2&m3&m4
print(np.sum(m), ' of ', len(m))

s82 = s82[m]

6975  of  9246


In [57]:
s82.rename_column()

lcname,length_sdss,sigmaEXP_sdss,sigmaMAP_sdss,tauEXP_sdss,tauMAP_sdss,length_sdss-ps1,sigmaEXP_sdss-ps1,sigmaMAP_sdss-ps1,tauEXP_sdss-ps1,tauMAP_sdss-ps1,dbID,ra_1,dec_1,SDR5ID_1,M_i_1,M_i_corr,redshift_1,mass_BH_1,Lbol,u,g,r,i,z,Au,SDSS_NAME,RA,DEC,REDSHIFT,LOGLBOL,LOGBH,LOGBH_ERR,LOGEDD_RATIO,MI_Z2,SDR5ID_2,ra_2,dec_2,redshift_2,M_i_2,mass_BH_2,chi2_pdf,log10tau,log10sigmahat,log10tau_lim_lo,log10tau_lim_hi,log10sigmahat_lim_lo,log10sigmahat_lim_hi,edge,Plike,Pnoise,Pinf,mu,Npts,SFinf,tauMAP_chelsea,sigmaMAP_chelsea
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str18,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64
70.txt,3336.9332799999975,0.06622421705058883,0.05984751944546515,122.6490220515577,43.947398211560085,5147.007797999999,--,0.1414213562373095,--,575.0000000000001,70,2.169302,1.238649,301,-23.901,-24.181,1.073,0.0,0.0,20.793,20.469,20.197,20.04,20.0,0.116,000840.63+011419.1,2.1693060398101807,1.2386499643325806,1.073699951171875,45.52781425110629,7.523330610633552,1.5230929717266002,-0.09588690464482141,-24.28560773968867,301,2.169302,1.238649,1.073,-23.901,0.0,1.1926,1.90162,-0.686379,1.70316,3.08136,-0.957464,-0.627884,0,138.926,117.218,136.073,20.2857,60,0.09622425731393176,79.72967613938569,0.06804082486132039
98.txt,2582.962741999996,0.17768456506773989,0.1425442932431927,926.094822794734,346.4224029622469,4767.016951999998,0.17310108214083317,0.15534946096966964,366.7595534657186,195.0728544419519,98,1.091028,0.962126,144,-23.399,-23.576,0.7867,0.0,0.0,20.79,20.183,19.849,19.818,19.43,0.183,000421.84+005743.6,1.0910290479660034,0.9621239900588989,0.792900025844574,45.421472462631726,8.817950339204,0.1672667771274661,-1.4968484216898403,-23.96421289067481,144,1.091028,0.962126,0.7867,-23.399,0.0,1.27857,2.72909,-0.7177,2.6183,4.47248,-0.840168,-0.660262,0,123.092,75.9121,122.524,19.8702,52,0.23211252059810764,535.9077035848122,0.1641283373132241
233.txt,3321.0338420000044,0.10075751425165866,0.08457207262730722,656.8411333494078,193.68268108139895,5505.090184000001,0.08836165515962953,0.08481144613166788,23.17606014197748,15.432798620779016,233,0.331289,0.17723,58,-24.735,-25.058,1.6199,0.0,0.0,20.892,20.554,20.431,20.199,20.099,0.154,000119.50+001038.0,0.33128899335861206,0.17722800374031067,1.62090003490448,45.95221191379889,8.921718166686233,0.20424216150370847,-1.0698767980049055,-25.008960685706665,58,0.331289,0.17723,1.6199,-24.735,0.0,1.19407,2.44022,-0.848145,2.24434,4.1202,-1.02269,-0.781607,0,134.36,107.71,132.865,20.3141,57,0.12325897950662115,275.5624262891833,0.08715726025126551
1018.txt,3336.9333019999976,0.19658989986620978,0.15925866404367295,717.7566916392584,289.67763034974274,5511.071033,0.21944178126621033,0.2084173212910267,2.326795898528782,34.18145321920661,1018,1.364696,-0.098956,190,-23.121,-24.045,0.6125,0.0,45.433,20.098,19.722,19.784,19.485,19.541,0.178,000527.52-000556.2,1.3646990060806274,-0.09895599633455276,0.6123999953269958,45.39391039241853,9.022211072036765,0.2674947222920013,-1.7286712247357983,-24.03915568655799,190,1.364696,-0.098956,0.6125,-23.121,0.0,1.12426,2.87294,-0.627733,2.72995,4.49924,-0.719803,-0.591494,0,152.593,82.2158,152.094,19.4742,62,0.33696948003053717,746.3456398263817,0.23827340438249772
1310.txt,3321.033924000003,0.04259839263300376,0.03745187919140339,383.23865805003277,145.46098372293594,5505.090513000003,0.16782713150619089,0.14034153752344863,1950.7661616699622,982.1410477550927,1310,0.221552,-0.292485,36,-26.451,-26.974,2.7563,9.361,46.76,20.707,19.663,19.61,19.705,19.529,0.174,000053.17-001732.9,0.22155599296092987,-0.29249000549316406,2.7571001052856445,46.78928392693668,9.249395337439456,0.05766163885177335

In [56]:
s82_ = np.load('../code/s82drw_shen2011_dic.npy').item()
s82_['r']

SDR5ID,ra,dec,redshift,M_i,mass_BH,chi2_pdf,log10tau,log10sigmahat,log10tau_lim_lo,log10tau_lim_hi,log10sigmahat_lim_lo,log10sigmahat_lim_hi,edge,Plike,Pnoise,Pinf,mu,Npts,lambdaRF,SFinf,tauRF,SFinf4000,tau4000,d2d_asec_DR7,SDSS_NAME,RA,DEC,REDSHIFT,LOGLBOL,LOGBH,LOGBH_ERR,LOGEDD_RATIO,MI_Z2
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,bytes18,float64,float64,float64,float64,float64,float64,float64,float64
301,2.169302,1.238649,1.073,-23.901,0.0,1.1926,1.90162,-0.686379,1.70316,3.08136,-0.957464,-0.627884,0,138.926,117.218,136.073,20.2857,60,3014.954172696575,0.09622425731393176,38.46101116226999,0.08403753294740945,40.35461188965733,0.014948619064117841,000840.63+011419.1,2.1693060398101807,1.2386499643325806,1.073699951171875,45.52781425110629,7.523330610633552,1.5230929717266002,-0.09588690464482141,-24.28560773968867
144,1.091028,0.962126,0.7867,-23.399,0.0,1.27857,2.72909,-0.7177,2.6183,4.47248,-0.840168,-0.660262,0,123.092,75.9121,122.524,19.8702,52,3498.0690658756366,0.23211252059810764,299.9427456119171,0.21767353283986823,306.8582023644366,0.008160006786109475,000421.84+005743.6,1.0910290479660034,0.9621239900588989,0.792900025844574,45.421472462631726,8.817950339204,0.1672667771274661,-1.4968484216898403,-23.96421289067481
58,0.331289,0.17723,1.6199,-24.735,0.0,1.19407,2.44022,-0.848145,2.24434,4.1202,-1.02269,-0.781607,0,134.36,107.71,132.865,20.3141,57,2385.587236154052,0.12325897950662115,105.18051310705879,0.0962276725781407,114.84029045008221,0.007186574652845284,000119.50+001038.0,0.33128899335861206,0.17722800374031067,1.62090003490448,45.95221191379889,8.921718166686233,0.20424216150370847,-1.0698767980049055,-25.008960685706665
190,1.364696,-0.098956,0.6125,-23.121,0.0,1.12426,2.87294,-0.627733,2.72995,4.49924,-0.719803,-0.591494,0,152.593,82.2158,152.094,19.4742,62,3875.968992248062,0.33696948003053717,462.8500091946553,0.33192349347557637,465.3351137599637,0.010821882163806102,000527.52-000556.2,1.3646990060806274,-0.09895599633455276,0.6123999953269958,45.39391039241853,9.022211072036765,0.2674947222920013,-1.7286712247357983,-24.03915568655799
36,0.221552,-0.292485,2.7563,-26.451,9.361,0.896713,2.53532,-1.15549,2.3373,4.25247,-1.32342,-1.07652,0,167.706,147.08,166.563,19.5994,56,1663.871362777201,0.06776782820446034,91.31870198192287,0.04451979857776681,106.00334532917908,0.023050764668535072,000053.17-001732.9,0.22155599296092987,-0.29249000549316406,2.7571001052856445,46.78928392693668,9.249395337439456,0.05766163885177335,-0.5604819556203395,-26.916981368954627
49,0.28137,0.686984,1.3988,-24.917,8.817,0.870333,2.5301,-0.832748,2.3953,4.21627,-0.94041,-0.795638,0,175.03,103.201,173.65,19.72,63,2605.46940136735,0.14163010195042328,141.2882219746354,0.11533953999461645,151.96918774877338,0.02101275011528123,000107.52+004113.1,0.2813670039176941,0.6869890093803406,1.3990000486373901,46.03569644938995,8.779701698389463,0.10077066846943966,-0.8443757941170773,-25.23082414221863
229,1.669492,0.643952,1.3383,-24.427,8.666,0.987443,2.3469,-0.714265,2.19097,3.9847,-0.839718,-0.670921,0,149.463,99.129,147.676,20.0716,61,2672.8820082966254,0.15067408672366103,95.06042909654711,0.12421531648481532,101.80366307444544,0.004962043940038134,000640.67+003838.2,1.6694910526275635,0.6439509987831116,1.3370000123977661,45.84014516210808,8.58543659024028,0.12310791899487153,-0.8456619732497614,-24.738376414675884
699,4.850273,-0.276395,1.769,-25.251,0.0,1.07542,2.90037,-0.789369,2.75814,4.4926,-0.908632,-0.723503,0,115.51,74.7377,114.929,20.2314,51,2257.1325388226796,0.23970084651702747,287.10915715783983,0.1822370449106084,316.4408499242961,0.039543166536742495,001924.06-001635.0,4.850282192230225,-0.27640101313591003,1.7651000022888184,46.002350093324324,9.122054257696913,0.1666785162004007,-1.2200747094901558,-25.535346152682955
578,4.026021,0.478127,1.8725,-24.999,0.0,1.05

In [58]:
# Fit Chelsea's and Celerite
# Fixing D=0, E=0, 
# i.e. only M_i  

# Use Shen2011 REDSHIFT,  MI_Z2,  LOGBH 

redshift_col = 'redshift_1'# 'REDSHIFT'
mi_col = 'M_i_corr'# 'MI_Z2'
mbh_col = 'mass_BH_1'# 'LOGBH'
lambdaobs = 6250 # for r-band, in angstroms 

cols = ['MAP_chelsea']#, 'EXP_sdss', 'EXP_sdss-ps1']
use_linmix = False

for i in range(len(cols)):
    print('Fitting results from ', cols[i])
    tau_col  = 'tau'+cols[i]
    sigma_col = 'sigma'+cols[i]
    var = 2 * (s82[sigma_col].data ** 2.0) # variance in mag^2, 
    # equal to sigmahat^2*tau/365 = SF_inf^2  = 2 * sigma^2 
    redshift = s82[redshift_col].data
    tauOBS  = s82[tau_col].data # obs frame , in days 
    tauRF = tauOBS / (1. + redshift) # correct to rest frame 
    mass_bh = s82[mbh_col].data 
    select = mass_bh > 0
    
    f_sf = np.log10(np.sqrt(var))[select] # f(sf) is log10(sfinf)
    f_tau = np.log10(tauRF)[select]  # f(tau) is a log10(tauRF ), 

    lRF = lambdaobs /  (1.+ s82[redshift_col]) # rest-frame wavelength in Angstroms
    lRF = lRF[select]
    M_i_corr = s82[mi_col].data[select] # i-band absolute magnitude 
    
    for f,B,name in zip([f_sf,f_tau],  
                     [-0.479, 0.17 ],
                    ['SF', 'tau']) :
        print(' running the linmix for  ', name  )
        x = (M_i_corr+23.)   
        xsig = 1e-5 
        # Equation 7, MacLeod+2010 
        # log(f) = A + B*log(lambdaRF) + C* log(M_i) + D*log(M_BH) + E* log(z)
        # we set D,E = 0
        # and first, remove wavelength trend: 
        # y = log(f) - B*log(lambdaRF) = A + C log(M_i)
        y = f - B*np.log10(lRF/4000.)  
        ysig = 0.01 

        # run the linmix for x1 :  M_i_corr ... 
        if use_linmix : 
            lm1 = linmix.LinMix(x=x, y=y, xsig=xsig, ysig=ysig, K=3)
            lm1.run_mcmc(silent=True)

            # store the MCMC chains 
            np.save('coeff_'+name+'_linmix_3_Shen2011_'+cols[i], lm1.chain[:])

            a = lm1.chain[:]['alpha']
            alpha1 = np.mean(a)
            alpha1_err = np.std(a)

            b = lm1.chain[:]['beta']
            beta1 = np.mean(b)
            beta1_err = np.std(b)

            print('alpha = ', alpha1, ' +/- ', alpha1_err)
            print('beta = ', beta1, ' +/- ', beta1_err)
        if not use_linmix:# 
             # Fit x1 and y1 :  
            # stack the latent variables x1,y1  ....
            X = np.vstack((x, y)).T
            dX = np.zeros((len(x), 2, 2))
            dX[:, 0, 0] = xsig ** 2
            dX[:, 1, 1] = ysig ** 2
            dX[:, 0, 1] = dX[:, 1, 0] = 0 # no cross-covariance ... 

            min_func = lambda beta: -TLS_logL(beta, X, dX)
            beta_fit = optimize.fmin(min_func, x0=[-1, 1])
            print(beta_fit)
            def get_m_b(beta):
                b = np.dot(beta, beta) / beta[1]
                m = -beta[0] / beta[1]
                return m, b

            slope, offset  = get_m_b(beta_fit)
            print('For band ', band, 'A=', offset, 'C=', slope)
           

Fitting results from  MAP_chelsea
 running the linmix for   SF
Optimization terminated successfully.
         Current function value: 61199303.547593
         Iterations: 117
         Function evaluations: 222
[-0.24558989 -1.37858719]
For band  r A= -1.4223380683469193 C= -0.17814607167390134
 running the linmix for   tau
Optimization terminated successfully.
         Current function value: 47059107.756684
         Iterations: 55
         Function evaluations: 105
[0.37662113 1.56447051]
For band  r A= 1.6551359985269383 C= -0.24073392749223232


Read the results : compare the posteriors for A,C  from linmix  for Chelsea's r-band, Celerite SDSS, and Celerite SDSS-PS1...

    Fitting results from  MAP_chelsea
     running the linmix for   SF
    alpha =  -1.600675689301655  +/-  0.05042471767439149
    beta =  -0.23845757550662638  +/-  0.01866650227304934
     running the linmix for   tau
    alpha =  1.4997470284386887  +/-  0.04317178309211009
    beta =  -0.28666653157584177  +/-  0.016178850580755102